<a href="https://colab.research.google.com/github/ShaliniAnandaPhD/Neuron/blob/main/Tutorial_7_CLI_Basics_Command_Line_Agent_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In previous tutorials, you've built intelligent agents with communication, memory, rules, monitoring, and configuration. Now we're adding command-line interfaces so you can manage, control, and interact with your agent systems from the terminal.

 What you'll build:

 • NeuroCommand CLI framework for agent management

 • Interactive command-line interface with auto-completion

 • Agent lifecycle management (start, stop, status, list)

 • Real-time monitoring commands and system inspection

 • Configuration management through CLI

 • Batch operations and scripting capabilities

 Why this matters:
 Production AI systems need operational tools for deployment, monitoring, and
 troubleshooting. Command-line interfaces provide powerful, scriptable access
 for DevOps teams, system administrators, and developers working with agent systems.

 By the end, you'll understand:

 • How to build professional CLI tools for AI systems

 • Command parsing, validation, and execution patterns

 • Interactive terminal interfaces with real-time updates

 • Operational workflows for agent system management

 • CLI design patterns for complex distributed systems

In [1]:
print("Tutorial 7: CLI Basics - Command Line Agent Management")
print("=" * 57)
print()
print("Building powerful command-line interfaces for agent system management...")
print()

Tutorial 7: CLI Basics - Command Line Agent Management

Building powerful command-line interfaces for agent system management...



In [2]:
# Essential imports
import uuid
import time
import threading
import queue
import json
import os
import sys
import copy
import shlex
import argparse
import cmd
import readline
from abc import ABC, abstractmethod
from dataclasses import dataclass, field, asdict
from typing import Any, Dict, List, Optional, Set, Callable, Union, Tuple
from enum import Enum
from collections import defaultdict
from pathlib import Path
import subprocess

In [3]:
# Import our foundation from previous tutorials
AgentID = str
MessageID = str

class MessagePriority(Enum):
    LOW = 1
    NORMAL = 2
    HIGH = 3
    URGENT = 4

@dataclass
class Message:
    id: MessageID
    sender: AgentID
    recipients: List[AgentID]
    content: Any
    priority: MessagePriority = MessagePriority.NORMAL
    metadata: Dict[str, Any] = field(default_factory=dict)
    created_at: float = field(default_factory=time.time)

    @classmethod
    def create(cls, sender: AgentID, recipients: List[AgentID], content: Any,
               priority: MessagePriority = MessagePriority.NORMAL) -> 'Message':
        return cls(
            id=str(uuid.uuid4()),
            sender=sender,
            recipients=recipients,
            content=content,
            priority=priority
        )

class CommandResult:
    """
    Result of executing a CLI command

    This standardizes command return values and provides
    consistent error handling and output formatting.
    """

    def __init__(self, success: bool = True, message: str = "", data: Any = None,
                 exit_code: int = 0):
        self.success = success
        self.message = message
        self.data = data
        self.exit_code = exit_code
        self.timestamp = time.time()

    @classmethod
    def success(cls, message: str = "Operation completed successfully", data: Any = None) -> 'CommandResult':
        """Create a successful command result"""
        return cls(success=True, message=message, data=data, exit_code=0)

    @classmethod
    def error(cls, message: str, exit_code: int = 1, data: Any = None) -> 'CommandResult':
        """Create an error command result"""
        return cls(success=False, message=message, data=data, exit_code=exit_code)

    def __str__(self) -> str:
        status = "✅" if self.success else "❌"
        return f"{status} {self.message}"

class CLICommand:
    """
    Base class for CLI commands

    This provides a framework for building structured, documented
    commands with argument parsing and help text.
    """

    def __init__(self, name: str, description: str, help_text: str = ""):
        self.name = name
        self.description = description
        self.help_text = help_text or description
        self.parser = argparse.ArgumentParser(
            prog=name,
            description=description,
            add_help=False  # We'll handle help ourselves
        )
        self._setup_arguments()

    def _setup_arguments(self):
        """Override this to add command-specific arguments"""
        pass

    def execute(self, args: List[str], context: 'CLIContext') -> CommandResult:
        """Execute the command with given arguments"""
        try:
            # Parse arguments
            parsed_args = self.parser.parse_args(args)

            # Execute the command logic
            return self._execute_impl(parsed_args, context)

        except SystemExit:
            # argparse calls sys.exit on error, we catch it
            return CommandResult.error("Invalid arguments. Use 'help <command>' for usage.")
        except Exception as e:
            return CommandResult.error(f"Command execution failed: {e}")

    def _execute_impl(self, args: argparse.Namespace, context: 'CLIContext') -> CommandResult:
        """Override this to implement command logic"""
        return CommandResult.error("Command not implemented")

    def get_help(self) -> str:
        """Get formatted help text for this command"""
        help_text = f"{self.name} - {self.description}\n\n"
        help_text += self.parser.format_help()
        return help_text

class CLIContext:
    """
    Context object passed to CLI commands

    This provides commands with access to the agent system,
    configuration, and shared state.
    """

    def __init__(self):
        self.agents: Dict[AgentID, 'ManagedAgent'] = {}
        self.agent_manager: Optional['AgentManager'] = None
        self.config: Dict[str, Any] = {}
        self.history: List[str] = []
        self.variables: Dict[str, Any] = {}
        self.debug_mode = False
        self.output_format = "table"  # table, json, yaml

    def add_agent(self, agent: 'ManagedAgent'):
        """Add an agent to the context"""
        self.agents[agent.id] = agent

    def remove_agent(self, agent_id: AgentID) -> bool:
        """Remove an agent from the context"""
        if agent_id in self.agents:
            del self.agents[agent_id]
            return True
        return False

    def get_agent(self, identifier: str) -> Optional['ManagedAgent']:
        """Get an agent by ID or name"""
        # Try by ID first
        if identifier in self.agents:
            return self.agents[identifier]

        # Try by name
        for agent in self.agents.values():
            if agent.name.lower() == identifier.lower():
                return agent

        return None

    def list_agents(self) -> List['ManagedAgent']:
        """Get all agents"""
        return list(self.agents.values())

class ManagedAgent:
    """
    An agent that can be managed through the CLI

    This wraps our previous agent classes with management
    capabilities and status tracking for CLI operations.
    """

    def __init__(self, agent_id: Optional[AgentID] = None, name: str = "",
                 agent_type: str = "basic"):
        self.id = agent_id or str(uuid.uuid4())
        self.name = name or f"agent_{self.id[:8]}"
        self.agent_type = agent_type
        self.status = "stopped"  # stopped, starting, running, stopping, error
        self.created_at = time.time()
        self.started_at: Optional[float] = None
        self.stopped_at: Optional[float] = None

        # Message processing
        self._message_queue = queue.Queue()
        self._stop_event = threading.Event()
        self._processing_thread = None

        # Statistics
        self.messages_processed = 0
        self.messages_sent = 0
        self.errors = 0
        self.last_activity = None

        print(f"🤖 Created ManagedAgent: {self.name} ({self.id[:8]}...)")

    def start(self) -> bool:
        """Start the agent"""
        if self.status == "running":
            return False

        try:
            self.status = "starting"
            self._stop_event.clear()
            self._processing_thread = threading.Thread(
                target=self._processing_loop,
                daemon=True,
                name=f"ManagedAgent-{self.name}"
            )
            self._processing_thread.start()
            self.status = "running"
            self.started_at = time.time()

            print(f"▶️  Started agent: {self.name}")
            return True

        except Exception as e:
            self.status = "error"
            print(f"❌ Failed to start agent {self.name}: {e}")
            return False

    def stop(self) -> bool:
        """Stop the agent"""
        if self.status != "running":
            return False

        try:
            self.status = "stopping"
            self._stop_event.set()

            if self._processing_thread:
                self._processing_thread.join(timeout=2.0)
                if self._processing_thread.is_alive():
                    print(f"⚠️  Warning: Agent {self.name} thread didn't stop cleanly")

            self.status = "stopped"
            self.stopped_at = time.time()

            print(f"⏹️  Stopped agent: {self.name}")
            return True

        except Exception as e:
            self.status = "error"
            print(f"❌ Failed to stop agent {self.name}: {e}")
            return False

    def send_message(self, recipients: List[AgentID], content: Any) -> Message:
        """Send a message"""
        message = Message.create(
            sender=self.id,
            recipients=recipients,
            content=content
        )

        self.messages_sent += 1
        self.last_activity = time.time()

        return message

    def receive_message(self, message: Message):
        """Receive a message for processing"""
        self._message_queue.put(message)
        self.last_activity = time.time()

    def process_message(self, message: Message):
        """Process a received message"""
        # Simple echo behavior for demo
        response = f"Echo from {self.name}: {message.content}"

        # Simulate processing time
        time.sleep(0.01)

        self.messages_processed += 1
        return response

    def _processing_loop(self):
        """Main message processing loop"""
        while not self._stop_event.is_set():
            try:
                message = self._message_queue.get(timeout=0.1)
                self.process_message(message)
                self._message_queue.task_done()

            except queue.Empty:
                continue
            except Exception as e:
                self.errors += 1
                print(f"❌ Error in {self.name}: {e}")

    def get_status(self) -> Dict[str, Any]:
        """Get detailed status information"""
        uptime = None
        if self.started_at and self.status == "running":
            uptime = time.time() - self.started_at

        return {
            'id': self.id,
            'name': self.name,
            'type': self.agent_type,
            'status': self.status,
            'uptime': uptime,
            'messages_processed': self.messages_processed,
            'messages_sent': self.messages_sent,
            'errors': self.errors,
            'queue_size': self._message_queue.qsize(),
            'last_activity': self.last_activity,
            'created_at': self.created_at,
            'started_at': self.started_at,
            'stopped_at': self.stopped_at
        }

# =============================================================================
# CLI COMMAND IMPLEMENTATIONS
# =============================================================================

class ListCommand(CLICommand):
    """List all agents in the system"""

    def __init__(self):
        super().__init__("list", "List all agents in the system")

    def _setup_arguments(self):
        self.parser.add_argument(
            "--status",
            choices=["running", "stopped", "error", "all"],
            default="all",
            help="Filter agents by status"
        )
        self.parser.add_argument(
            "--format",
            choices=["table", "json", "brief"],
            default="table",
            help="Output format"
        )

    def _execute_impl(self, args: argparse.Namespace, context: CLIContext) -> CommandResult:
        agents = context.list_agents()

        # Filter by status
        if args.status != "all":
            agents = [a for a in agents if a.status == args.status]

        if not agents:
            return CommandResult.success("No agents found matching criteria")

        if args.format == "json":
            data = [agent.get_status() for agent in agents]
            output = json.dumps(data, indent=2, default=str)

        elif args.format == "brief":
            output = "\n".join([f"{a.name} ({a.status})" for a in agents])

        else:  # table format
            output = self._format_table(agents)

        return CommandResult.success(f"Found {len(agents)} agents", output)

    def _format_table(self, agents: List[ManagedAgent]) -> str:
        """Format agents as a table"""
        if not agents:
            return "No agents to display"

        # Table headers
        headers = ["Name", "ID", "Status", "Type", "Messages", "Errors", "Uptime"]

        # Calculate column widths
        col_widths = [len(h) for h in headers]

        rows = []
        for agent in agents:
            status = agent.get_status()
            uptime_str = ""
            if status['uptime']:
                uptime_str = f"{status['uptime']:.1f}s"

            row = [
                agent.name,
                agent.id[:8] + "...",
                agent.status,
                agent.agent_type,
                str(agent.messages_processed),
                str(agent.errors),
                uptime_str
            ]
            rows.append(row)

            # Update column widths
            for i, cell in enumerate(row):
                col_widths[i] = max(col_widths[i], len(cell))

        # Build table
        separator = "+" + "+".join(["-" * (w + 2) for w in col_widths]) + "+"

        output = [separator]

        # Header row
        header_row = "|" + "|".join([f" {h:<{col_widths[i]}} " for i, h in enumerate(headers)]) + "|"
        output.append(header_row)
        output.append(separator)

        # Data rows
        for row in rows:
            data_row = "|" + "|".join([f" {cell:<{col_widths[i]}} " for i, cell in enumerate(row)]) + "|"
            output.append(data_row)

        output.append(separator)

        return "\n".join(output)

class StartCommand(CLICommand):
    """Start one or more agents"""

    def __init__(self):
        super().__init__("start", "Start one or more agents")

    def _setup_arguments(self):
        self.parser.add_argument(
            "agents",
            nargs="+",
            help="Agent names or IDs to start"
        )
        self.parser.add_argument(
            "--wait",
            action="store_true",
            help="Wait for agents to fully start before returning"
        )

    def _execute_impl(self, args: argparse.Namespace, context: CLIContext) -> CommandResult:
        results = []

        for agent_identifier in args.agents:
            agent = context.get_agent(agent_identifier)

            if not agent:
                results.append(f"❌ Agent not found: {agent_identifier}")
                continue

            if agent.status == "running":
                results.append(f"⚠️  Agent {agent.name} is already running")
                continue

            success = agent.start()
            if success:
                results.append(f"✅ Started agent: {agent.name}")

                if args.wait:
                    # Wait a moment for startup
                    time.sleep(0.1)
                    if agent.status == "running":
                        results.append(f"✅ Agent {agent.name} is now running")
                    else:
                        results.append(f"⚠️  Agent {agent.name} startup may have failed")
            else:
                results.append(f"❌ Failed to start agent: {agent.name}")

        output = "\n".join(results)
        return CommandResult.success("Start operation completed", output)

class StopCommand(CLICommand):
    """Stop one or more agents"""

    def __init__(self):
        super().__init__("stop", "Stop one or more agents")

    def _setup_arguments(self):
        self.parser.add_argument(
            "agents",
            nargs="+",
            help="Agent names or IDs to stop"
        )
        self.parser.add_argument(
            "--force",
            action="store_true",
            help="Force stop agents even if they're busy"
        )

    def _execute_impl(self, args: argparse.Namespace, context: CLIContext) -> CommandResult:
        results = []

        for agent_identifier in args.agents:
            agent = context.get_agent(agent_identifier)

            if not agent:
                results.append(f"❌ Agent not found: {agent_identifier}")
                continue

            if agent.status != "running":
                results.append(f"⚠️  Agent {agent.name} is not running")
                continue

            success = agent.stop()
            if success:
                results.append(f"✅ Stopped agent: {agent.name}")
            else:
                results.append(f"❌ Failed to stop agent: {agent.name}")

        output = "\n".join(results)
        return CommandResult.success("Stop operation completed", output)

class StatusCommand(CLICommand):
    """Show detailed status of agents"""

    def __init__(self):
        super().__init__("status", "Show detailed status of one or more agents")

    def _setup_arguments(self):
        self.parser.add_argument(
            "agents",
            nargs="*",
            help="Agent names or IDs (if none specified, shows all)"
        )
        self.parser.add_argument(
            "--refresh",
            type=int,
            metavar="SECONDS",
            help="Refresh status every N seconds"
        )

    def _execute_impl(self, args: argparse.Namespace, context: CLIContext) -> CommandResult:
        if args.agents:
            agents = []
            for identifier in args.agents:
                agent = context.get_agent(identifier)
                if agent:
                    agents.append(agent)
                else:
                    return CommandResult.error(f"Agent not found: {identifier}")
        else:
            agents = context.list_agents()

        if not agents:
            return CommandResult.success("No agents to show status for")

        if args.refresh:
            return self._refresh_status(agents, args.refresh)
        else:
            output = self._format_detailed_status(agents)
            return CommandResult.success("Agent status", output)

    def _format_detailed_status(self, agents: List[ManagedAgent]) -> str:
        """Format detailed status for agents"""
        output = []

        for agent in agents:
            status = agent.get_status()

            output.append(f"\n📊 Agent: {agent.name}")
            output.append(f"   ID: {agent.id}")
            output.append(f"   Type: {agent.agent_type}")
            output.append(f"   Status: {self._status_with_emoji(agent.status)}")

            if status['uptime']:
                output.append(f"   Uptime: {status['uptime']:.1f} seconds")

            output.append(f"   Messages Processed: {agent.messages_processed}")
            output.append(f"   Messages Sent: {agent.messages_sent}")
            output.append(f"   Errors: {agent.errors}")
            output.append(f"   Queue Size: {status['queue_size']}")

            if agent.last_activity:
                last_active = time.time() - agent.last_activity
                output.append(f"   Last Activity: {last_active:.1f} seconds ago")
            else:
                output.append("   Last Activity: Never")

        return "\n".join(output)

    def _status_with_emoji(self, status: str) -> str:
        """Add emoji to status"""
        emoji_map = {
            "running": "🟢 running",
            "stopped": "🔴 stopped",
            "starting": "🟡 starting",
            "stopping": "🟡 stopping",
            "error": "❌ error"
        }
        return emoji_map.get(status, f"❓ {status}")

    def _refresh_status(self, agents: List[ManagedAgent], interval: int) -> CommandResult:
        """Show refreshing status display"""
        try:
            while True:
                # Clear screen
                os.system('clear' if os.name == 'posix' else 'cls')

                print(f"🔄 Agent Status (refreshing every {interval}s) - Press Ctrl+C to stop\n")
                print(self._format_detailed_status(agents))

                time.sleep(interval)

        except KeyboardInterrupt:
            return CommandResult.success("Status monitoring stopped")

class CreateCommand(CLICommand):
    """Create a new agent"""

    def __init__(self):
        super().__init__("create", "Create a new agent")

    def _setup_arguments(self):
        self.parser.add_argument(
            "name",
            help="Name for the new agent"
        )
        self.parser.add_argument(
            "--type",
            default="basic",
            choices=["basic", "smart", "analytical"],
            help="Type of agent to create"
        )
        self.parser.add_argument(
            "--start",
            action="store_true",
            help="Start the agent immediately after creation"
        )

    def _execute_impl(self, args: argparse.Namespace, context: CLIContext) -> CommandResult:
        # Check if agent name already exists
        if context.get_agent(args.name):
            return CommandResult.error(f"Agent with name '{args.name}' already exists")

        # Create new agent
        agent = ManagedAgent(
            name=args.name,
            agent_type=args.type
        )

        context.add_agent(agent)

        result_msg = f"Created agent: {agent.name} (ID: {agent.id[:8]}...)"

        if args.start:
            if agent.start():
                result_msg += " and started"
            else:
                result_msg += " but failed to start"

        return CommandResult.success(result_msg)

class DeleteCommand(CLICommand):
    """Delete an agent"""

    def __init__(self):
        super().__init__("delete", "Delete an agent")

    def _setup_arguments(self):
        self.parser.add_argument(
            "agent",
            help="Agent name or ID to delete"
        )
        self.parser.add_argument(
            "--force",
            action="store_true",
            help="Force delete even if agent is running"
        )

    def _execute_impl(self, args: argparse.Namespace, context: CLIContext) -> CommandResult:
        agent = context.get_agent(args.agent)

        if not agent:
            return CommandResult.error(f"Agent not found: {args.agent}")

        if agent.status == "running" and not args.force:
            return CommandResult.error(f"Agent {agent.name} is running. Use --force to delete anyway.")

        # Stop the agent if running
        if agent.status == "running":
            agent.stop()

        # Remove from context
        context.remove_agent(agent.id)

        return CommandResult.success(f"Deleted agent: {agent.name}")

class SendCommand(CLICommand):
    """Send a message to an agent"""

    def __init__(self):
        super().__init__("send", "Send a message to one or more agents")

    def _setup_arguments(self):
        self.parser.add_argument(
            "recipients",
            nargs="+",
            help="Agent names or IDs to send message to"
        )
        self.parser.add_argument(
            "message",
            help="Message content to send"
        )
        self.parser.add_argument(
            "--priority",
            choices=["low", "normal", "high", "urgent"],
            default="normal",
            help="Message priority"
        )

    def _execute_impl(self, args: argparse.Namespace, context: CLIContext) -> CommandResult:
        # Resolve recipient agents
        recipients = []
        for identifier in args.recipients:
            agent = context.get_agent(identifier)
            if agent:
                recipients.append(agent)
            else:
                return CommandResult.error(f"Recipient not found: {identifier}")

        # Convert priority
        priority_map = {
            "low": MessagePriority.LOW,
            "normal": MessagePriority.NORMAL,
            "high": MessagePriority.HIGH,
            "urgent": MessagePriority.URGENT
        }
        priority = priority_map[args.priority]

        # Create and send message
        message = Message.create(
            sender="cli_user",
            recipients=[agent.id for agent in recipients],
            content=args.message,
            priority=priority
        )

        # Deliver to each recipient
        for agent in recipients:
            agent.receive_message(message)

        recipient_names = [agent.name for agent in recipients]
        return CommandResult.success(f"Message sent to: {', '.join(recipient_names)}")

class HelpCommand(CLICommand):
    """Show help information"""

    def __init__(self):
        super().__init__("help", "Show help information")

    def _setup_arguments(self):
        self.parser.add_argument(
            "command",
            nargs="?",
            help="Show help for specific command"
        )

    def _execute_impl(self, args: argparse.Namespace, context: CLIContext) -> CommandResult:
        # This will be implemented by the CLI itself
        return CommandResult.success("Help will be provided by CLI")

class NeuroCommandCLI(cmd.Cmd):
    """
    Interactive command-line interface for agent management

    This provides a professional CLI experience with command completion,
    history, and interactive features for managing agent systems.
    """

    intro = """
🚀 NeuroCommand CLI - Agent Management Interface
Type 'help' for available commands or 'help <command>' for specific help.
Type 'exit' or 'quit' to leave.
"""

    prompt = "neuro> "

    def __init__(self):
        super().__init__()
        self.context = CLIContext()
        self.commands: Dict[str, CLICommand] = {}
        self.running = True

        # Register built-in commands
        self._register_commands()

        # Setup readline for better experience
        self._setup_readline()

    def _register_commands(self):
        """Register all available commands"""
        commands = [
            ListCommand(),
            StartCommand(),
            StopCommand(),
            StatusCommand(),
            CreateCommand(),
            DeleteCommand(),
            SendCommand(),
            HelpCommand()
        ]

        for command in commands:
            self.commands[command.name] = command

    def _setup_readline(self):
        """Setup readline for command history and completion"""
        try:
            # Enable tab completion
            readline.set_completer_delims(' \t\n`!@#$%^&*()=+[{]}\\|;:\'",<>?')
            readline.parse_and_bind("tab: complete")

            # Load command history if available
            history_file = os.path.expanduser("~/.neuro_history")
            if os.path.exists(history_file):
                readline.read_history_file(history_file)

        except ImportError:
            # readline not available on all systems
            pass

    def _save_history(self):
        """Save command history"""
        try:
            history_file = os.path.expanduser("~/.neuro_history")
            readline.write_history_file(history_file)
        except:
            pass

    def default(self, line: str):
        """Handle unknown commands"""
        if line.strip():
            parts = shlex.split(line)
            command_name = parts[0]
            args = parts[1:]

            if command_name in self.commands:
                result = self.commands[command_name].execute(args, self.context)

                if result.success:
                    if result.data:
                        print(result.data)
                    else:
                        print(result.message)
                else:
                    print(f"❌ {result.message}")
            else:
                print(f"❌ Unknown command: {command_name}")
                print("Type 'help' for available commands.")

    def do_help(self, arg: str):
        """Show help information"""
        if not arg:
            print("\nAvailable commands:")
            print("=" * 40)

            for name, command in sorted(self.commands.items()):
                print(f"  {name:<12} - {command.description}")

            print("\nBuilt-in commands:")
            print(f"  {'exit':<12} - Exit the CLI")
            print(f"  {'quit':<12} - Exit the CLI")
            print(f"  {'clear':<12} - Clear the screen")

            print("\nType 'help <command>' for detailed help on a specific command.")
        else:
            if arg in self.commands:
                print(self.commands[arg].get_help())
            else:
                print(f"❌ Unknown command: {arg}")

    def do_exit(self, arg: str):
        """Exit the CLI"""
        return self.do_quit(arg)

    def do_quit(self, arg: str):
        """Exit the CLI"""
        print("👋 Goodbye!")
        self._save_history()
        self.running = False
        return True

    def do_clear(self, arg: str):
        """Clear the screen"""
        os.system('clear' if os.name == 'posix' else 'cls')

    def completenames(self, text: str, *ignored) -> List[str]:
        """Complete command names"""
        commands = list(self.commands.keys()) + ['exit', 'quit', 'clear', 'help']
        return [cmd for cmd in commands if cmd.startswith(text)]

    def complete_start(self, text: str, line: str, begidx: int, endidx: int) -> List[str]:
        """Complete agent names for start command"""
        return self._complete_agent_names(text, status_filter="stopped")

    def complete_stop(self, text: str, line: str, begidx: int, endidx: int) -> List[str]:
        """Complete agent names for stop command"""
        return self._complete_agent_names(text, status_filter="running")

    def complete_status(self, text: str, line: str, begidx: int, endidx: int) -> List[str]:
        """Complete agent names for status command"""
        return self._complete_agent_names(text)

    def complete_delete(self, text: str, line: str, begidx: int, endidx: int) -> List[str]:
        """Complete agent names for delete command"""
        return self._complete_agent_names(text)

    def complete_send(self, text: str, line: str, begidx: int, endidx: int) -> List[str]:
        """Complete agent names for send command"""
        return self._complete_agent_names(text, status_filter="running")

    def _complete_agent_names(self, text: str, status_filter: str = None) -> List[str]:
        """Helper to complete agent names with optional status filtering"""
        agents = self.context.list_agents()

        if status_filter:
            agents = [a for a in agents if a.status == status_filter]

        names = [agent.name for agent in agents]
        return [name for name in names if name.startswith(text)]

    def emptyline(self):
        """Handle empty line input"""
        pass  # Don't repeat last command

    def cmdloop(self, intro=None):
        """Main command loop with error handling"""
        if intro is not None:
            self.intro = intro

        if self.intro:
            self.stdout.write(str(self.intro) + "\n")

        stop = None
        while not stop and self.running:
            try:
                stop = self.onecmd_plus_hooks(self.cmdqueue.pop(0) if self.cmdqueue else self.readline())
            except KeyboardInterrupt:
                print("\nUse 'exit' or 'quit' to leave the CLI.")
            except EOFError:
                print("\n👋 Goodbye!")
                stop = True
            except Exception as e:
                print(f"❌ Unexpected error: {e}")

        self._save_history()

    def readline(self):
        """Read a line with the prompt"""
        try:
            return input(self.prompt)
        except EOFError:
            return 'EOF'

class BatchCLI:
    """
    Non-interactive CLI for batch operations and scripting

    This provides a simpler interface for running commands
    programmatically or from scripts.
    """

    def __init__(self):
        self.context = CLIContext()
        self.commands: Dict[str, CLICommand] = {}
        self._register_commands()

    def _register_commands(self):
        """Register all available commands"""
        commands = [
            ListCommand(),
            StartCommand(),
            StopCommand(),
            StatusCommand(),
            CreateCommand(),
            DeleteCommand(),
            SendCommand()
        ]

        for command in commands:
            self.commands[command.name] = command

    def execute_command(self, command_line: str) -> CommandResult:
        """Execute a single command"""
        try:
            parts = shlex.split(command_line.strip())
            if not parts:
                return CommandResult.success("No command provided")

            command_name = parts[0]
            args = parts[1:]

            if command_name in self.commands:
                return self.commands[command_name].execute(args, self.context)
            else:
                return CommandResult.error(f"Unknown command: {command_name}")

        except Exception as e:
            return CommandResult.error(f"Command execution failed: {e}")

    def execute_script(self, script_path: str) -> List[CommandResult]:
        """Execute commands from a script file"""
        results = []

        try:
            with open(script_path, 'r') as f:
                for line_num, line in enumerate(f, 1):
                    line = line.strip()

                    # Skip empty lines and comments
                    if not line or line.startswith('#'):
                        continue

                    print(f"Executing: {line}")
                    result = self.execute_command(line)
                    results.append(result)

                    if not result.success:
                        print(f"❌ Line {line_num}: {result.message}")
                        break
                    else:
                        print(f"✅ Line {line_num}: {result.message}")

        except FileNotFoundError:
            error_result = CommandResult.error(f"Script file not found: {script_path}")
            results.append(error_result)
        except Exception as e:
            error_result = CommandResult.error(f"Error executing script: {e}")
            results.append(error_result)

        return results

# =============================================================================
# INITIALIZATION COMPLETE
# =============================================================================

print("🔧 Tutorial 7 initialization complete!")
print("✅ All classes loaded successfully:")
print("   - CommandResult for standardized command responses")
print("   - CLICommand base class for structured commands")
print("   - CLIContext for shared state management")
print("   - ManagedAgent for CLI-controllable agents")
print("   - Complete command implementations (list, start, stop, etc.)")
print("   - NeuroCommandCLI for interactive terminal interface")
print("   - BatchCLI for scripting and automation")
print()
print("🚀 Ready to build powerful command-line agent management tools!")
print()

🔧 Tutorial 7 initialization complete!
✅ All classes loaded successfully:
   - CommandResult for standardized command responses
   - CLICommand base class for structured commands
   - CLIContext for shared state management
   - ManagedAgent for CLI-controllable agents
   - Complete command implementations (list, start, stop, etc.)
   - NeuroCommandCLI for interactive terminal interface
   - BatchCLI for scripting and automation

🚀 Ready to build powerful command-line agent management tools!



In [4]:
# DEMO SECTION: Let's build and use our CLI!
# =============================================================================

print("=" * 57)
print("🚀 Tutorial 7: CLI Basics - Command Line Agent Management")
print("=" * 57)
print()


🚀 Tutorial 7: CLI Basics - Command Line Agent Management



In [5]:
# Step 1: Create batch CLI and demonstrate basic commands
print("📝 Step 1: Creating batch CLI and testing basic commands...")

batch_cli = BatchCLI()

# Create some test agents
print("   Creating test agents...")
commands = [
    "create alice --type smart --start",
    "create bob --type analytical",
    "create charlie --type basic --start"
]

for command in commands:
    print(f"   $ {command}")
    result = batch_cli.execute_command(command)
    if result.success:
        print(f"     ✅ {result.message}")
    else:
        print(f"     ❌ {result.message}")

print()

📝 Step 1: Creating batch CLI and testing basic commands...
   Creating test agents...
   $ create alice --type smart --start
🤖 Created ManagedAgent: alice (791ba426...)
▶️  Started agent: alice
     ✅ Created agent: alice (ID: 791ba426...) and started
   $ create bob --type analytical
🤖 Created ManagedAgent: bob (f48aa1f1...)
     ✅ Created agent: bob (ID: f48aa1f1...)
   $ create charlie --type basic --start
🤖 Created ManagedAgent: charlie (d216b687...)
▶️  Started agent: charlie
     ✅ Created agent: charlie (ID: d216b687...) and started



In [6]:
# Step 2: Test agent management commands
print("📝 Step 2: Testing agent management commands...")

management_commands = [
    "list",
    "start bob",
    "list --status running",
    "status alice bob",
    "send alice charlie 'Hello from the CLI!'"
]

for command in management_commands:
    print(f"   $ {command}")
    result = batch_cli.execute_command(command)
    if result.success:
        if result.data:
            # Print first few lines of data if it's long
            lines = str(result.data).split('\n')
            if len(lines) > 10:
                print("     " + "\n     ".join(lines[:10]))
                print(f"     ... ({len(lines) - 10} more lines)")
            else:
                print("     " + "\n     ".join(lines))
        else:
            print(f"     ✅ {result.message}")
    else:
        print(f"     ❌ {result.message}")
    print()


📝 Step 2: Testing agent management commands...
   $ list
     +---------+-------------+---------+------------+----------+--------+--------+
     | Name    | ID          | Status  | Type       | Messages | Errors | Uptime |
     +---------+-------------+---------+------------+----------+--------+--------+
     | alice   | 791ba426... | running | smart      | 0        | 0      | 16.4s  |
     | bob     | f48aa1f1... | stopped | analytical | 0        | 0      |        |
     | charlie | d216b687... | running | basic      | 0        | 0      | 16.3s  |
     +---------+-------------+---------+------------+----------+--------+--------+

   $ start bob
▶️  Started agent: bob
     ✅ Started agent: bob

   $ list --status running
     +---------+-------------+---------+------------+----------+--------+--------+
     | Name    | ID          | Status  | Type       | Messages | Errors | Uptime |
     +---------+-------------+---------+------------+----------+--------+--------+
     | alice   | 791

In [7]:
# Step 3: Demonstrate scripting capabilities
print("📝 Step 3: Testing CLI scripting capabilities...")

# Create a test script
script_content = """# Agent Management Script
create david --type smart
create emma --type analytical
start david emma
list --status running
send david emma "Hello from script!"
status --refresh 2
stop david emma
"""

script_path = "/tmp/test_script.txt"
try:
    with open(script_path, 'w') as f:
        f.write(script_content)

    print(f"   Created test script: {script_path}")
    print("   Script contents:")
    for i, line in enumerate(script_content.strip().split('\n'), 1):
        print(f"     {i:2}: {line}")

    print("\n   Executing script...")
    # Execute first few commands (skip refresh command for demo)
    script_lines = [
        "create david --type smart",
        "create emma --type analytical",
        "start david emma",
        "list --status running"
    ]

    for line in script_lines:
        print(f"   $ {line}")
        result = batch_cli.execute_command(line)
        print(f"     {'✅' if result.success else '❌'} {result.message}")

except Exception as e:
    print(f"   ❌ Error with script demo: {e}")

print()


📝 Step 3: Testing CLI scripting capabilities...
   Created test script: /tmp/test_script.txt
   Script contents:
      1: # Agent Management Script
      2: create david --type smart
      3: create emma --type analytical
      4: start david emma
      5: list --status running
      6: send david emma "Hello from script!"
      7: status --refresh 2
      8: stop david emma

   Executing script...
   $ create david --type smart
🤖 Created ManagedAgent: david (34bdf9ff...)
     ✅ Created agent: david (ID: 34bdf9ff...)
   $ create emma --type analytical
🤖 Created ManagedAgent: emma (5a4b6882...)
     ✅ Created agent: emma (ID: 5a4b6882...)
   $ start david emma
▶️  Started agent: david
▶️  Started agent: emma
     ✅ Start operation completed
   $ list --status running
     ✅ Found 5 agents



In [8]:
# Step 4: Test advanced CLI features
print("📝 Step 4: Testing advanced CLI features...")

# Test error handling
print("   Testing error handling...")
error_commands = [
    "start nonexistent_agent",
    "delete alice",  # Should fail because alice is running
    "send nobody 'test message'",
    "invalid_command"
]

for command in error_commands:
    print(f"   $ {command}")
    result = batch_cli.execute_command(command)
    print(f"     ❌ {result.message}")

print()


📝 Step 4: Testing advanced CLI features...
   Testing error handling...
   $ start nonexistent_agent
     ❌ Start operation completed
   $ delete alice
     ❌ Agent alice is running. Use --force to delete anyway.
   $ send nobody 'test message'
     ❌ Recipient not found: nobody
   $ invalid_command
     ❌ Unknown command: invalid_command



In [12]:
# Step 5: Create interactive CLI demo
print("📝 Step 5: Demonstrating interactive CLI features...")

def run_interactive_demo():
    """Run a brief interactive CLI demo"""
    cli = NeuroCommandCLI()

    # Add some agents to the context
    for agent_name, agent_type in [("Alice", "smart"), ("Bob", "analytical"), ("Charlie", "basic")]:
        agent = ManagedAgent(name=agent_name, agent_type=agent_type)
        cli.context.add_agent(agent)
        agent.start()

    print("   Interactive CLI created with 3 test agents")
    print("   Available commands: list, start, stop, status, create, delete, send, help")
    print("   You can test the interactive CLI by running: cli.cmdloop()")
    print("   For now, we'll simulate some interactive commands...")

    # Simulate interactive commands
    demo_commands = [
        "help",
        "list",
        "status Alice",
        "send Alice 'Hello from interactive CLI!'"
    ]

    for command in demo_commands:
        print(f"\n   neuro> {command}")
        cli.default(command)

    # Clean up
    for agent in cli.context.list_agents():
        agent.stop()

    return cli

interactive_cli = run_interactive_demo()
print()


📝 Step 5: Demonstrating interactive CLI features...
🤖 Created ManagedAgent: Alice (e26d3d28...)
▶️  Started agent: Alice
🤖 Created ManagedAgent: Bob (ca4cf609...)
▶️  Started agent: Bob
🤖 Created ManagedAgent: Charlie (c00c757e...)
▶️  Started agent: Charlie
   Interactive CLI created with 3 test agents
   Available commands: list, start, stop, status, create, delete, send, help
   You can test the interactive CLI by running: cli.cmdloop()
   For now, we'll simulate some interactive commands...

   neuro> help
Help will be provided by CLI

   neuro> list
+---------+-------------+---------+------------+----------+--------+--------+
| Name    | ID          | Status  | Type       | Messages | Errors | Uptime |
+---------+-------------+---------+------------+----------+--------+--------+
| Alice   | e26d3d28... | running | smart      | 0        | 0      | 0.0s   |
| Bob     | ca4cf609... | running | analytical | 0        | 0      | 0.0s   |
| Charlie | c00c757e... | running | basic      | 

In [13]:
# Step 6: Performance and monitoring
print("📝 Step 6: CLI performance analysis...")

# Test command execution performance
import time

commands_to_test = ["list", "status", "list --format json"]
performance_results = {}

for command in commands_to_test:
    start_time = time.time()

    # Execute command multiple times
    for _ in range(100):
        batch_cli.execute_command(command)

    total_time = time.time() - start_time
    avg_time = total_time / 100
    performance_results[command] = avg_time

print("   Command execution performance (100 iterations):")
for command, avg_time in performance_results.items():
    print(f"     {command:<20}: {avg_time*1000:.2f}ms avg")

# Test memory usage with many agents
print("\n   Testing scalability with many agents...")
start_time = time.time()

for i in range(50):
    result = batch_cli.execute_command(f"create agent_{i:03d} --type basic")
    if not result.success:
        print(f"   ❌ Failed to create agent_{i:03d}: {result.message}")
        break

creation_time = time.time() - start_time
agent_count = len(batch_cli.context.list_agents())

print(f"   Created {agent_count} agents in {creation_time:.2f} seconds")
print(f"   Average creation time: {creation_time/agent_count*1000:.2f}ms per agent")

print()

📝 Step 6: CLI performance analysis...
   Command execution performance (100 iterations):
     list                : 0.10ms avg
     status              : 0.09ms avg
     list --format json  : 0.15ms avg

   Testing scalability with many agents...
🤖 Created ManagedAgent: agent_000 (d25afb0f...)
🤖 Created ManagedAgent: agent_001 (d5bd4ca1...)
🤖 Created ManagedAgent: agent_002 (339f1baa...)
🤖 Created ManagedAgent: agent_003 (6e732dd1...)
🤖 Created ManagedAgent: agent_004 (7938d0bd...)
🤖 Created ManagedAgent: agent_005 (5995dcc0...)
🤖 Created ManagedAgent: agent_006 (2adba048...)
🤖 Created ManagedAgent: agent_007 (6c3a2548...)
🤖 Created ManagedAgent: agent_008 (d470a40a...)
🤖 Created ManagedAgent: agent_009 (9b3bfed4...)
🤖 Created ManagedAgent: agent_010 (48f4f549...)
🤖 Created ManagedAgent: agent_011 (ad8ac9ab...)
🤖 Created ManagedAgent: agent_012 (02f961b2...)
🤖 Created ManagedAgent: agent_013 (570eefd1...)
🤖 Created ManagedAgent: agent_014 (cbb5a4bb...)
🤖 Created ManagedAgent: agent_015

In [14]:
# Step 7: Visualization of CLI usage
print("📝 Step 7: Creating CLI usage visualization...")

try:
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    # Prepare CLI usage data
    agents = batch_cli.context.list_agents()

    # Agent status distribution
    status_counts = {}
    for agent in agents:
        status = agent.status
        status_counts[status] = status_counts.get(status, 0) + 1

    # Agent type distribution
    type_counts = {}
    for agent in agents:
        agent_type = agent.agent_type
        type_counts[agent_type] = type_counts.get(agent_type, 0) + 1

    # Performance data
    command_names = list(performance_results.keys())
    command_times = [t * 1000 for t in performance_results.values()]  # Convert to ms

    # Create CLI dashboard
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Agent Status Distribution',
            'Agent Type Distribution',
            'Command Performance',
            'System Timeline'
        ),
        specs=[[{"type": "domain"}, {"type": "domain"}],
               [{"type": "bar"}, {"type": "scatter"}]]
    )

    # Agent status pie chart
    fig.add_trace(
        go.Pie(
            labels=list(status_counts.keys()),
            values=list(status_counts.values()),
            name="Status"
        ),
        row=1, col=1
    )

    # Agent type pie chart
    fig.add_trace(
        go.Pie(
            labels=list(type_counts.keys()),
            values=list(type_counts.values()),
            name="Types"
        ),
        row=1, col=2
    )

    # Command performance bar chart
    fig.add_trace(
        go.Bar(
            x=command_names,
            y=command_times,
            name='Execution Time',
            marker_color='lightblue',
            text=[f"{t:.1f}ms" for t in command_times],
            textposition='auto'
        ),
        row=2, col=1
    )

    # System growth timeline (simulated)
    timeline_points = list(range(len(agents)))
    fig.add_trace(
        go.Scatter(
            x=timeline_points,
            y=timeline_points,  # Cumulative agent count
            mode='lines+markers',
            name='Agent Count Growth',
            line=dict(color='green', width=3),
            marker=dict(size=6)
        ),
        row=2, col=2
    )

    # Update layout
    fig.update_layout(
        title_text="NeuroCommand CLI - System Dashboard",
        height=800,
        showlegend=True,
        template='plotly_white'
    )

    # Update axes
    fig.update_xaxes(title_text="Commands", row=2, col=1)
    fig.update_yaxes(title_text="Time (ms)", row=2, col=1)

    fig.update_xaxes(title_text="Creation Order", row=2, col=2)
    fig.update_yaxes(title_text="Total Agents", row=2, col=2)

    fig.show()

    print("   ✅ CLI dashboard visualization created!")
    print("   📊 The dashboard shows:")
    print("      - Current agent status distribution")
    print("      - Agent type breakdown")
    print("      - Command execution performance")
    print("      - System growth over time")
    print()

except ImportError:
    print("   ⚠️  Plotly not available - skipping visualization")
    print("   💡 To see CLI visualizations, install plotly: pip install plotly")
    print("   📊 CLI summary:")
    print(f"      Total agents created: {len(batch_cli.context.list_agents())}")
    print(f"      Running agents: {len([a for a in batch_cli.context.list_agents() if a.status == 'running'])}")
    print(f"      Command performance: {min(performance_results.values())*1000:.1f}-{max(performance_results.values())*1000:.1f}ms")
    print()


📝 Step 7: Creating CLI usage visualization...


   ✅ CLI dashboard visualization created!
   📊 The dashboard shows:
      - Current agent status distribution
      - Agent type breakdown
      - Command execution performance
      - System growth over time



In [15]:
# Step 8: Clean up and final analysis
print("📝 Step 8: Final system analysis and cleanup...")

# Get final statistics
final_agents = batch_cli.context.list_agents()
running_agents = [a for a in final_agents if a.status == "running"]
stopped_agents = [a for a in final_agents if a.status == "stopped"]

print("   Final CLI System State:")
print(f"     Total agents created: {len(final_agents)}")
print(f"     Running agents: {len(running_agents)}")
print(f"     Stopped agents: {len(stopped_agents)}")

if running_agents:
    print("     Running agent details:")
    for agent in running_agents[:5]:  # Show first 5
        status = agent.get_status()
        uptime = status.get('uptime', 0) or 0
        print(f"       {agent.name}: {uptime:.1f}s uptime, {agent.messages_processed} messages")

    if len(running_agents) > 5:
        print(f"       ... and {len(running_agents) - 5} more")

# Calculate total system utilization
total_messages = sum(a.messages_processed for a in final_agents)
total_errors = sum(a.errors for a in final_agents)
error_rate = total_errors / max(total_messages, 1) * 100

print(f"\n   System Performance:")
print(f"     Total messages processed: {total_messages}")
print(f"     Total errors: {total_errors}")
print(f"     Error rate: {error_rate:.2f}%")
print(f"     Commands tested: {len(performance_results)}")

# Stop all running agents
print("\n   Stopping all running agents...")
stop_count = 0
for agent in final_agents:
    if agent.status == "running":
        if agent.stop():
            stop_count += 1

print(f"   ✅ Stopped {stop_count} agents")

print("\n✅ Tutorial 7 Complete!")
print()


📝 Step 8: Final system analysis and cleanup...
   Final CLI System State:
     Total agents created: 55
     Running agents: 5
     Stopped agents: 50
     Running agent details:
       alice: 271.8s uptime, 1 messages
       bob: 255.5s uptime, 0 messages
       charlie: 271.8s uptime, 1 messages
       david: 234.3s uptime, 0 messages
       emma: 234.3s uptime, 0 messages

   System Performance:
     Total messages processed: 2
     Total errors: 0
     Error rate: 0.00%
     Commands tested: 3

   Stopping all running agents...
⏹️  Stopped agent: alice
⏹️  Stopped agent: bob
⏹️  Stopped agent: charlie
⏹️  Stopped agent: david
⏹️  Stopped agent: emma
   ✅ Stopped 5 agents

✅ Tutorial 7 Complete!



In [16]:
# SUMMARY OF WHAT WE LEARNED
# =============================================================================

print("📚 WHAT WE LEARNED:")

print("=" * 40)

print("1. 🖥️  Built a comprehensive CLI framework")

print("   - Structured command system with argument parsing")

print("   - Interactive terminal interface with auto-completion")

print("   - Batch processing and scripting capabilities")

print("   - Professional error handling and user feedback")

print()

print("2. 🛠️  Implemented production-ready agent management")

print("   - Agent lifecycle control (create, start, stop, delete)")

print("   - Real-time status monitoring and system inspection")

print("   - Message sending and communication testing")

print("   - Bulk operations and filtering capabilities")

print()

print("3. 🚀 Created enterprise-grade operational tools")

print("   - Command history and auto-completion")

print("   - Scriptable automation for DevOps workflows")

print("   - Performance monitoring and optimization")

print("   - Scalability testing with large agent populations")

print()

print("4. 📊 Added comprehensive CLI analytics")

print("   - Command execution performance tracking")

print("   - System utilization and error rate monitoring")

print("   - Visual dashboards for operational insights")

print("   - Growth and capacity planning metrics")

print()


📚 WHAT WE LEARNED:
1. 🖥️  Built a comprehensive CLI framework
   - Structured command system with argument parsing
   - Interactive terminal interface with auto-completion
   - Batch processing and scripting capabilities
   - Professional error handling and user feedback

2. 🛠️  Implemented production-ready agent management
   - Agent lifecycle control (create, start, stop, delete)
   - Real-time status monitoring and system inspection
   - Message sending and communication testing
   - Bulk operations and filtering capabilities

3. 🚀 Created enterprise-grade operational tools
   - Command history and auto-completion
   - Scriptable automation for DevOps workflows
   - Performance monitoring and optimization
   - Scalability testing with large agent populations

4. 📊 Added comprehensive CLI analytics
   - Command execution performance tracking
   - System utilization and error rate monitoring
   - Visual dashboards for operational insights
   - Growth and capacity planning metrics

